In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from itertools import cycle
from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
colors = [("red", "red"), ("green", "green"), ("blue", "blue"), ("orange", "orange")]
color_cycle = cycle(colors)

In [4]:
program = "graph_coloring"
# program = "dijkstra_token_ring"
# program = "maximal_matching"
# graph_name = "graph_powerlaw_cluster_graph_n7"
# graph_name = "implicit_graph_n8"
graph_name = "graph_7"
sched = 0
no_simulations = 100000
me = False
fault_interval = 1

duong_mode = True

selected_nodes = [9]

In [5]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}"

In [6]:
# filenames = [
#     get_filename(
#         graph_name,
#         sched,
#         SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
#         "",
#         no_simulations,
#         me,
#         fault_interval,
#     )
# ]
filenames = []
if duong_mode:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )
else:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )


In [7]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

In [8]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(211)

In [9]:
bins = np.linspace(0, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [10]:
bins

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [11]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([   0, 8385, 7458, 6589, 5974, 5502, 5039, 4494, 4117, 3868, 3488,
       3273, 3077, 2749, 2460, 2339, 2242, 2058, 1913, 1744, 1598, 1493,
       1402, 1296, 1227, 1140, 1074,  989,  918,  816,  779,  712,  670,
        593,  576,  554,  490,  501,  460,  393,  387,  366,  291,  304,
        303,  306,  248,  239,  198,  203,  200,  170,  158,  153,  138,
        117,  118,  109,  106,   97,   91,   83,   66,   66,   77,   69,
         52,   56,   44,   38,   49,   47,   46,   41,   40,   30,   34,
         32,   17,   27,   21,   22,   20,   11,   14,   18,   21,   16,
          7,   14,   10,    7,   13,   11,   12,   11,   14,    5,   11,
          6,    4,    6,    2,    2,    9,    3,    7,    8,    3,    2,
          1,    2,    2,    1,    0,    2,    0,    1,    2,    1,    0,
          0,    0,    1,    0,    2,    0,    0,    0,    2,    0,    0,
          0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,    0,    0,    0,    0,    0,   

In [12]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [13]:
hist_df[0]

,Steps,Count
0,0.0,0
1,1.0,8385
2,2.0,7458
3,3.0,6589
4,4.0,5974
...,...,...
206,206.0,0
207,207.0,0
208,208.0,0
209,209.0,0


In [14]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count
0,0.0,0
1,1.0,8385
2,2.0,7458
3,3.0,6589
4,4.0,5974
...,...,...
206,206.0,0
207,207.0,0
208,208.0,0
209,209.0,0


In [15]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count
Steps,
0.0,0
1.0,8385
2.0,7458
3.0,6589
4.0,5974
...,...
206.0,0
207.0,0
208.0,0


In [16]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [17]:
def get_filename():
    return f"{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}{'__duong' if duong_mode else ''}"

In [18]:
from scipy.optimize import curve_fit


def exp_func(x, a, b, c):
    return a * np.exp(-b * x) + c


# Fit the curve
popt, _ = curve_fit(exp_func, df_merged.index, df_merged["Count"], p0= [10, 0.1, 0])
y_fit = exp_func(df_merged.index, popt[0], popt[1], popt[2])
df_merged[f"exp_fit"] = y_fit

/home/agaru/anaconda3/envs/cvf/lib/python3.13/site-packages/pandas/core/indexes/base.py:952: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*new_inputs, **kwargs)


In [19]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 0:
        line_style = "solid"
        line.set_color(next(color_cycle)[0])
    else:
        line_style = "dashed"
        line.set_color("goldenrod")
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

# labels = ["Random Fault"]
labels = []
labels.extend(
    [
        f'Controlled {"(duong)" if duong_mode else ""} at node {n}'
        for n in selected_nodes
    ]
)
labels.append(f"$y = {popt[0]:.2f}e^{{{-popt[1]:.2f}x}} + {popt[1]:.2f}$")

custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
file_path = os.path.join(
    plots_dir,
    program,
    f"{get_filename()}.png",
)


plt.savefig(
    file_path,
    bbox_inches="tight",
)
plt.close()

print(f"Saved plot(s) for {file_path}")

Saved plot(s) for plots/graph_coloring/graph_coloring__graph_7__0__100000__1__9__duong.png


In [20]:
df_merged

,Count,exp_fit
Steps,,
0.0,0,6419.256840
1.0,8385,6024.680462
2.0,7458,5654.264227
3.0,6589,5306.528797
4.0,5974,4980.085413
...,...,...
206.0,0,-24.834306
207.0,0,-24.835183
208.0,0,-24.836007


In [21]:
# Save to file
cols = [
    f'Controlled {"(duong)" if duong_mode else ""} at node {n}' for n in selected_nodes
]
cols.append("exp_curve")
# df_merged.index = df_merged.index.astype(int)
df_merged.to_csv(os.path.join("results", program, f"w_exp_{get_filename()}.csv"))